<a href="https://colab.research.google.com/github/MK-Sindhu/A2_dl_ed24s401/blob/main/Tentative_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sasukess1_nature_12k_path = kagglehub.dataset_download('sasukess1/nature-12k')



In [4]:
import wandb

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [6]:
!ls /kaggle/input/nature-12k/inaturalist_12K/train

Amphibia  Arachnida  Fungi    Mammalia	Plantae
Animalia  Aves	     Insecta  Mollusca	Reptilia


### ANSWER 1

In [7]:
# transformations for training and testing
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))

])
transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])



train_dataset = datasets.ImageFolder(root='/kaggle/input/nature-12k/inaturalist_12K/train', transform=transform_train)

test_dataset = datasets.ImageFolder(root='/kaggle/input/nature-12k/inaturalist_12K/val', transform=transform_test)



# Creating Dataloaders

train_loader= DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
train_loader= DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)



In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
# !pip install thop

In [10]:
Xfrozen_modules='off'

In [11]:
class FlexibleCNN(nn.Module):
  def __init__(
      self,
      in_channels= 3,
      conv_channels=[32, 64, 128, 256, 512],
      kernel_size= 3,
      activation =nn.ReLU,
      num_classes=10,
      init_image_size=224,
      dense_neurons=512
    ):

    super(FlexibleCNN, self).__init__()

    self.activation=activation()

    # Building Conv Layers

    conv_layers =[]
    current_in_channels =in_channels
    current_image_size=init_image_size

    for out_channels in conv_channels:

      # Conv layer
      conv_layers.append(nn.Conv2d(
          in_channels=current_in_channels,
          out_channels=out_channels,
          kernel_size=kernel_size,
          stride=1,
          padding= kernel_size // 2


      ))

      # Activation
      conv_layers.append(activation())

      # Max Pooling
      conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

      # Update for next iteration
      current_in_channels = out_channels
      current_image_size = current_image_size // 2

    # Sequential Combination of all conc layers
    self.conv_layers =nn.Sequential(*conv_layers)

    # Flattening
    flattened_dim = current_in_channels * current_image_size**2

    self.fc1 = nn.Linear(flattened_dim, dense_neurons)
    self.fc2 = nn.Linear(dense_neurons, num_classes)


  def forward(self, x):

    # Pass through all conv + pool blocks

    x= self.conv_layers(x)

    # FLatten
    x = x.view(x.size(0), -1)

    # Fully layers with activation
    x=self.activation(self.fc1(x))
    x=self.fc2(x)

    return x


model = FlexibleCNN(
    in_channels=3,
    conv_channels=[32, 64, 128, 256, 512],
    kernel_size=3,
    activation=nn.ReLU,
    num_classes=len(train_dataset.classes),  # For iNaturalist or any other dataset
    init_image_size=224,  # If your images are resized to 224x224
    dense_neurons=1024     # Example size for the penultimate layer
)

#print(type(model))
#
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# print(type(model))

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print("Total trainable parameters:", total_params)

from thop import profile

# Create a dummy input tensor with batch size 1 and appropriate dimensions
dummy_input = torch.randn(1, 3, 224, 224).to(device)
macs, params = profile(model, inputs=(dummy_input,), verbose=False)
print("Total MACs:", macs)


Total trainable parameters: 27269962
Total MACs: 993896448.0


#### QUESTION 2

In [12]:
from torch.utils.data import random_split

In [ ]:
print(next(model.parameters()).device)


cuda:0


In [13]:
# 1. Setup & Directory Check
# -------------------------
# !ls /kaggle/input/nature-12k  # List dataset structure

import wandb
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

In [14]:
# 2. Device Setup (GPU)
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [15]:
# 3. Define Data Transforms
# -------------------------
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# -------------------------
# 4. Load and Split Dataset
# -------------------------
dataset_path = "/kaggle/input/nature-12k/inaturalist_12K/train"
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform_train)
print("Total images in full_dataset:", len(full_dataset))
print("Classes:", full_dataset.classes)

# Split 80% train, 20% validation
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])
print("Training samples:", len(train_subset))
print("Validation samples:", len(val_subset))


Total images in full_dataset: 9999
Classes: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Training samples: 8000
Validation samples: 1999


In [ ]:

# -------------------------
# 5. Define the FlexibleCNN Model
# -------------------------
class FlexibleCNN(nn.Module):
    def __init__(
        self,
        in_channels=3,
        conv_channels=[32, 64, 128, 256, 512],
        kernel_size=3,
        activation=nn.ReLU,
        num_classes=10,
        init_image_size=224,
        dense_neurons=512,
        dropout=0.0
    ):
        super(FlexibleCNN, self).__init__()
        self.activation_fn = activation()
        self.dropout_fn = nn.Dropout(dropout)

        conv_layers = []
        current_in_channels = in_channels
        current_img_size = init_image_size

        for out_channels in conv_channels:
            conv_layers.append(nn.Conv2d(
                in_channels=current_in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=1,
                padding=kernel_size // 2
            ))
            conv_layers.append(activation())
            conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            current_in_channels = out_channels
            current_img_size //= 2

        self.conv_layers = nn.Sequential(*conv_layers)
        flattened_dim = current_in_channels * (current_img_size ** 2)
        self.fc1 = nn.Linear(flattened_dim, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, num_classes)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout_fn(self.activation_fn(self.fc1(x)))
        x = self.fc2(x)
        return x

# -------------------------
# 6. Define Training Function for One Run
# -------------------------
def train_and_validate():
    # Initialize wandb run; reinit=True ensures multiple runs can be started sequentially
    wandb.init(project="inaturalist_sweep", reinit=True)
    config = wandb.config

    # Map activation function string to actual PyTorch class
    act_fn_map = {
        'ReLU': nn.ReLU,
        'GELU': nn.GELU,
        'Silu': nn.SiLU
    }

    # Build model with hyperparameters from wandb.config
    model = FlexibleCNN(
        in_channels=3,
        conv_channels=config.conv_channels,
        kernel_size=3,
        activation=act_fn_map[config.activation_fn],
        num_classes=len(full_dataset.classes),
        init_image_size=224,
        dense_neurons=1024,
        dropout=config.dropout
    ).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

    # Create DataLoaders for train and validation subsets
    train_loader = DataLoader(train_subset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=config.batch_size, shuffle=False)

    # Training loop
    for epoch in range(config.epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100. * correct / total

        # Validation loop
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100. * val_correct / val_total

        # Print per-epoch metrics
        print(f"Epoch {epoch+1}/{config.epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })

    wandb.finish()

# -------------------------
# 7. Define Sweep Configuration and Launch Sweep Agent
# -------------------------
sweep_config = {
    'method': 'bayes',  # Options: 'grid', 'random', or 'bayes'
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.0001,
            'max': 0.01
        },
        'batch_size': {
            'values': [32, 64]
        },
        'activation_fn': {
            'values': ['ReLU', 'GELU', 'Silu']
        },
        'conv_channels': {
            'values': [[32, 64, 128, 256, 512], [64, 64, 64, 64, 64]]
        },
        'epochs': {
            'value': 10  # Fixed number of epochs per run
        },
        'dropout': {
            'values': [0.0, 0.2, 0.3]
        }
    }
}

# Initialize sweep and get a sweep ID
sweep_id = wandb.sweep(sweep_config, project="inaturalist_sweep")
print("Sweep ID:", sweep_id)

# Launch the sweep agent with wandb.agent (this will run the training function for the given count of runs)
wandb.agent(sweep_id, function=train_and_validate, count=10)


inaturalist_12K
Using device: cuda
Total images in full_dataset: 9999
Classes: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Training samples: 8000
Validation samples: 1999


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: hrhva050
Sweep URL: https://wandb.ai/ed24s401-indian-institute-of-technology-madras/inaturalist_sweep/sweeps/hrhva050
Sweep ID: hrhva050


...

wandb: Agent Starting Run: iif8nj3n with config:
wandb: 	activation_fn: ReLU
wandb: 	batch_size: 32
wandb: 	conv_channels: [64, 64, 64, 64, 64]
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	learning_rate: 0.00925183680775473


Epoch 1/10 | Train Loss: 2.7003, Train Acc: 8.91% | Val Loss: 2.3043, Val Acc: 9.35%
Epoch 2/10 | Train Loss: 2.3042, Train Acc: 9.79% | Val Loss: 2.3032, Val Acc: 10.61%
Epoch 3/10 | Train Loss: 2.3041, Train Acc: 9.84% | Val Loss: 2.3064, Val Acc: 8.85%
Epoch 4/10 | Train Loss: 2.3045, Train Acc: 9.44% | Val Loss: 2.3040, Val Acc: 9.45%
Epoch 5/10 | Train Loss: 2.3043, Train Acc: 9.47% | Val Loss: 2.3032, Val Acc: 9.90%
Epoch 6/10 | Train Loss: 2.3039, Train Acc: 9.81% | Val Loss: 2.3042, Val Acc: 9.35%
Epoch 7/10 | Train Loss: 2.3041, Train Acc: 9.60% | Val Loss: 2.3033, Val Acc: 10.21%
Epoch 8/10 | Train Loss: 2.3043, Train Acc: 9.97% | Val Loss: 2.3046, Val Acc: 10.26%
Epoch 9/10 | Train Loss: 2.3040, Train Acc: 9.74% | Val Loss: 2.3043, Val Acc: 10.26%
Epoch 10/10 | Train Loss: 2.3039, Train Acc: 9.99% | Val Loss: 2.3046, Val Acc: 9.45%


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▇▇▄▅▇▅█▆█
train_loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▃█▁▃▅▃▆▇▇▃
val_loss,▃▁█▃▁▃▁▄▃▄
epoch,9
train_accuracy,9.9875
train_loss,2.30386
val_accuracy,9.45473
val_loss,2.30456


wandb: Agent Starting Run: 1ixj5xov with config:
wandb: 	activation_fn: GELU
wandb: 	batch_size: 64
wandb: 	conv_channels: [32, 64, 128, 256, 512]
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0029849234842672947


Epoch 1/10 | Train Loss: 3.2266, Train Acc: 16.00% | Val Loss: 2.1669, Val Acc: 20.41%


In [ ]:
# import wandb
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, random_split
# from torchvision import datasets, transforms

# # Define transforms for training
# transform_train = transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# # Load the full dataset from Kaggle input
# full_dataset = datasets.ImageFolder(root="/kaggle/input/nature-12k/inaturalist_12K/train", transform=transform_train)

# # Split the dataset: 80% train, 20% validation
# val_size = int(0.2 * len(full_dataset))
# train_size = len(full_dataset) - val_size
# train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

# # Define the device (GPU if available)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("Using device:", device)

# # Sweep configuration as a Python dictionary
# sweep_config = {
#     'method': 'bayes',  # 'grid', 'random', or 'bayes'
#     'metric': {
#         'name': 'val_accuracy',
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'learning_rate': {
#             'min': 0.0001,
#             'max': 0.01
#         },
#         'batch_size': {
#             'values': [32, 64]
#         },
#         'activation_fn': {
#             'values': ['ReLU', 'GELU', 'Silu']  # Will map these to torch activations
#         },
#         'conv_channels': {
#             'values': [[32, 64, 128, 256, 512], [64, 64, 64, 64, 64]]
#         },
#         'epochs': {
#             'value': 10  # Fixed number of epochs for each run
#         },
#         'dropout': {
#             'values': [0.0, 0.2, 0.3]
#         }
#     }
# }

# # Initialize the sweep and obtain a sweep ID
# sweep_id = wandb.sweep(sweep_config, project='your_project_name')
# print("Sweep ID:", sweep_id)

# # Define the training function for each run of the sweep
# def train_and_validate():
#     # Initialize wandb for this run; configuration is available via wandb.config
#     wandb.init()

#     # Map string activations to actual PyTorch classes
#     act_fn_map = {
#         'ReLU': torch.nn.ReLU,
#         'GELU': torch.nn.GELU,
#         'Silu': torch.nn.SiLU
#     }

#     config = wandb.config  # Access hyperparameters

#     # Build your model (assuming FlexibleCNN is defined and imported)
#     model = FlexibleCNN(
#         in_channels=3,
#         conv_channels=config.conv_channels,
#         kernel_size=3,
#         activation=act_fn_map[config.activation_fn],
#         num_classes=len(full_dataset.classes),
#         init_image_size=224,
#         dense_neurons=1024  # You can also incorporate dropout if supported by your model
#     ).to(device)

#     # Define loss and optimizer
#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

#     # Create DataLoaders for training and validation subsets
#     train_loader = DataLoader(train_subset, batch_size=config.batch_size, shuffle=True)
#     val_loader = DataLoader(val_subset, batch_size=config.batch_size, shuffle=False)

#     # Training loop
#     for epoch in range(config.epochs):
#         model.train()
#         total_loss, correct, total = 0.0, 0, 0

#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()
#             _, predicted = outputs.max(1)
#             correct += predicted.eq(labels).sum().item()
#             total += labels.size(0)

#         train_loss_epoch = total_loss / len(train_loader)
#         train_accuracy = 100. * correct / total

#         # Validation loop
#         model.eval()
#         val_loss, val_correct, val_total = 0.0, 0, 0

#         with torch.no_grad():
#             for images, labels in val_loader:
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()
#                 _, predicted = outputs.max(1)
#                 val_correct += predicted.eq(labels).sum().item()
#                 val_total += labels.size(0)

#         val_loss_epoch = val_loss / len(val_loader)
#         val_accuracy = 100. * val_correct / val_total

#         # Print epoch details
#         print(f"Epoch {epoch+1}/{config.epochs} | Train Loss: {train_loss_epoch:.4f} | Train Acc: {train_accuracy:.2f}% | Val Loss: {val_loss_epoch:.4f} | Val Acc: {val_accuracy:.2f}%")

#         # Log metrics to wandb
#         wandb.log({
#             "epoch": epoch,
#             "train_loss": train_loss_epoch,
#             "train_accuracy": train_accuracy,
#             "val_loss": val_loss_epoch,
#             "val_accuracy": val_accuracy
#         })

#     wandb.finish()

# # Launch the agent: run the training function for a total of 10 runs
# wandb.agent(sweep_id, function=train_and_validate, count=10)


Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 